In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# importing numpy, matplotlib, seaborn and pandas
import importlib
from modules import classification_metrics,loader, crossval
modules_to_reload = [classification_metrics,loader, crossval]
for m in modules_to_reload:
    importlib.reload(m)


Traceback (most recent call last):
  File "/Users/kaustav/.vscode/extensions/ms-python.python-2025.16.0-darwin-arm64/python_files/python_server.py", line 134, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 3, in <module>
ModuleNotFoundError: No module named 'modules'



In [102]:
help(crossval)

Help on module modules.crossval in modules:

NAME
    modules.crossval

FUNCTIONS
    cv_score(model: str, model_instance, X_train, y_train, cv, scoring)
        Computed cross_val_score and returns
        - An array of Cross_Val score
        - Mean cross_val score
        - Std. dev. of cross_val scores

        Parameters:
            model:str - Name of the model passed as a string
            X_train - array like training data
            y_train - array like true labels

        Retuns:
            cv_score (Array)
            scores (pd.DataFrame)

FILE
    /Users/kaustav/Documents/Python/Machine learning/Projects/breastcancerpredict_notebook/modules/crossval.py




In [101]:
from my_packages.imports import *
from modules.loader import load_data
from modules.classification_metrics import classification_metrics
from modules.crossval import cv_score

In [69]:
df = load_data("data.csv")

from modules.dataexplore import *
none of these functions are working

Here some basic datatypes can be put into a dataexplore package. This same data_exploration can be used in different ml_projects. 

In [ ]:
object = df.select_dtypes(include='object').columns
number = df.select_dtypes(include='number').columns

Traceback (most recent call last):
  File "/Users/kaustav/.vscode/extensions/ms-python.python-2025.16.0-darwin-arm64/python_files/python_server.py", line 134, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [10]:
df = df.drop(['id','Unnamed: 32'], axis=1)

Encoding data and splitting into X,Y.

In [11]:
X = df.drop('diagnosis',axis=1)
y = df['diagnosis']
y = pd.get_dummies(y, drop_first=True, dtype=int)

Splitting the data into train and test split. This is the same code for all ML_projects so can be put inside a module.

In [12]:
from sklearn.model_selection import train_test_split 

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [15]:
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

modelling 

1.Logistic Regression   

In [18]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2')
lr.fit(X_train_scaled,y_train)
y_pred = lr.predict(X_test_scaled)

In [71]:
lr_metric, lr_cm = classification_metrics('Logistic Regression',y_test,y_pred)

In [72]:
lr_metric

,Model,Accuracy,F1 Score,Precision,Recall
0,Logistic Regression,0.965,0.952,0.976,0.93


In [73]:
lr_cm

,Pred 0,Pred 1
Actual 0,70,1
Actual 1,3,40


In [123]:
lr_cv, lr_cv_scores = cv_score("Logistic Regression", lr, X_train_scaled, y_train, cv=10, scoring='accuracy')

In [125]:
print(lr_cv_scores)

                 model  meanCV  stddevCV
0  Logistic Regression   0.974     0.019


Code 2 - Random Forest Classifier

In [108]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train_scaled,y_train)
y_pred = rfc.predict(X_test_scaled)

In [75]:
rfc_metric, rfc_cm = classification_metrics("Random Forrest Classifier", y_test,y_pred)

In [76]:
rfc_metric

,Model,Accuracy,F1 Score,Precision,Recall
0,Random Forrest Classifier,0.965,0.952,0.976,0.93


In [77]:
rfc_cm

,Pred 0,Pred 1
Actual 0,70,1
Actual 1,3,40


In [122]:
results = pd.concat([rfc_metric, lr_metric], ignore_index=True)
results

,Model,Accuracy,F1 Score,Precision,Recall
0,Random Forrest Classifier,0.965,0.952,0.976,0.93
1,Logistic Regression,0.965,0.952,0.976,0.93


In [126]:
rfc_cv, rfc_cv_scores = cv_score("Random Forrest",rfc, X_train_scaled, y_train, cv=10, scoring='accuracy')

In [127]:
print(rfc_cv_scores)

            model  meanCV  stddevCV
0  Random Forrest   0.963     0.024


In [128]:
pd.concat([rfc_cv_scores,lr_cv_scores], ignore_index=True).sort_values('meanCV', ascending=False)

,model,meanCV,stddevCV
1,Logistic Regression,0.974,0.019
0,Random Forrest,0.963,0.024


Logistic regression in this case does a better job than random forest classifier

Hyperparameter tuning

In [91]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    "penalty" : ['l1','l2','elasticnet'],
    "C": [0.25,0.5,0.75,1.0,1.5,1.75,2.0],
    "solver":['saga'],
    "l1_ratio":[0.1,0.5,0.9]
}
gridsearch = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, scoring='accuracy',cv=5)
gridsearch.fit(X_train_scaled,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.25, 0.5, 0.75, 1.0, 1.5, 1.75, 2.0],
                         'l1_ratio': [0.1, 0.5, 0.9],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['saga']},
             scoring='accuracy')

In [92]:
# Check best score and estimator found by random_search
best_score = gridsearch.best_score_
best_estimator = gridsearch.best_estimator_

print(f"Best Score:\n{best_score}\n")
print(f"Best Estimator:\n{best_estimator}")

Best Score:
0.9780219780219781

Best Estimator:
LogisticRegression(C=1.5, l1_ratio=0.1, solver='saga')


Final Model (Logistic Regression)

In [80]:
final_classifier = LogisticRegression(C=1.5,l1_ratio=0.1,solver='saga')
final_classifier.fit(X_train_scaled,y_train)
y_pred = final_classifier.predict(X_test_scaled)

In [81]:
final_metric, final_cm = classification_metrics('Logistic Refression Final',y_test,y_pred)

In [82]:
final_metric

,Model,Accuracy,F1 Score,Precision,Recall
0,Logistic Refression Final,0.991,0.988,1.0,0.977


In [83]:
final_cm

,Pred 0,Pred 1
Actual 0,71,0
Actual 1,1,42


In [85]:
result =pd.concat([lr_metric,rfc_metric, final_metric], ignore_index=True)
result

,Model,Accuracy,F1 Score,Precision,Recall
0,Logistic Regression,0.965,0.952,0.976,0.930
1,Random Forrest Classifier,0.965,0.952,0.976,0.930
2,Logistic Refression Final,0.991,0.988,1.000,0.977
